
 $V(h)=h(14−2h)(10−2h)=140h−48h^2+4h^3$<p></p>
 $V′(h)=140−96h+12h^2$<p></p>
 $V(1.9183)=120.1644$<p></p>

In [39]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
import numpy
import matplotlib.mlab as mlab

In [40]:
def main_func(x):
  return x*(14-2*x)*(10-2*x)

In [41]:
x = tf.Variable(tf.random_uniform([1], 0, 2))

In [43]:
loss = tf.reduce_mean(x*(14-2*x)*(10-2*x))  # Create an operation that calculates loss.
optimizer = tf.train.GradientDescentOptimizer(-0.01)  # Create an optimizer.
train = optimizer.minimize(loss)  # Create an operation that minimizes loss.
init = tf.initialize_all_variables() 

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [44]:
sess = tf.Session()
sess.run(init)

In [45]:
for step in range(10):
    sess.run(train)
    print(step, sess.run([x]))
print(sess.run([x]))

0 [array([1.615013], dtype=float32)]
1 [array([1.7775925], dtype=float32)]
2 [array([1.8502839], dtype=float32)]
3 [array([1.8848374], dtype=float32)]
4 [array([1.9017069], dtype=float32)]
5 [array([1.910047], dtype=float32)]
6 [array([1.9141954], dtype=float32)]
7 [array([1.9162651], dtype=float32)]
8 [array([1.9172993], dtype=float32)]
9 [array([1.9178164], dtype=float32)]
[array([1.9178164], dtype=float32)]


In [ ]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = 1000 - (x*y*z)
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 100
    child_num = 5

    space = OrderedDict((
        ('x', (10.,12.)),
        ('y', (5.,7.)),
        ('z', (0.,2.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 11.439790889773397, 6.124821171196882, 0.04416197499764851: 996.9057173195621
Eval 10.489817183683781, 5.545713883660466, 0.9104281233611471: 947.0371869937561
Eval 11.045425267176308, 6.3482996221886845, 0.2822197469739043: 980.2108447426659
Eval 10.666347952341724, 6.537040038716407, 1.4868265294484027: 896.3290224874901
Eval 10.226046991096398, 5.342626877288566, 1.9869112565596383: 891.4471827944391
Best [10.226046991096398, 5.342626877288566, 1.9869112565596383]

Eval 10.0, 5.0, 1.6801112584885427: 915.9944370755728
Eval 10.58007328971684, 5.0, 1.4513155878003068: 923.2248735728214
Eval 10.0, 7.0, 1.7411580885305775: 878.1189338028596
Eval 10.856013359001228, 5.0, 2.0: 891.4398664099878
Eval 10.882654565452523, 5.536039886447461, 2.0: 879.5063805104505
Best [10.0, 7.0, 1.7411580885305775]

Eval 10.0, 6.179975995527868, 1.2099270899530872: 925.2267962775103
Eval 10.18119107386847, 5.0, 2.0: 898.1880892613153
Eval 10.0, 6.999863020713381, 2.0: 860.0027395857323
Eval 10.30481047